In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime as dt
import random

In [2]:
data = pd.read_csv('data_small.csv')
df = data
df['day'] = pd.to_datetime(df.day)
df1 = data.groupby(by=['dns_qdomainname','day']).sum()

# df2 counts unique query names by domain name and day
df2 = data.groupby(by=['dns_qdomainname','day'])[['shieldid','dns_qname']].nunique()
df2 = pd.DataFrame(df2)
df2.rename(columns={'shieldid': 'num_shieldid', 'dns_qname': 'num_dns_qname'}, inplace=True)
    
# Concatenate df1 and df2 into df
df = pd.concat([df1, df2], axis=1)
df.reset_index(level=1, inplace=True)

In [3]:
def bin_and_merge(df, group_col, bin_col, bin_label):
    # Group by domain and sum the values for the given column
    group_by_domain = df.groupby(df.index)[bin_col].sum().reset_index()

    # Sort the data by ascending order
    group_by_domain.sort_values(by=bin_col, inplace=True)

    # Create the bin edges for 5 equal cuts
    bin_edges_a = pd.qcut(group_by_domain[bin_col], q=group_by_domain[bin_col].nunique(), duplicates='drop').sort_values().unique()

    # Create the bin labels
    bin_labels = [f"{int(bin_edges_a[i].left+1)}-{int(bin_edges_a[i].right)}" for i in range(len(bin_edges_a))]

    # Convert bin edges to floats
    bin_edges = [float(bin_edge.left) for bin_edge in bin_edges_a]
    bin_edges.append(float(bin_edges_a[-1].right))

    # Bin the data and assign the labels to each row
    group_by_domain['group'] = pd.cut(group_by_domain[bin_col], bins=bin_edges, labels=bin_labels)

    # Merge the labeled data back to the original dataframe
    df = pd.merge(df, group_by_domain[['dns_qdomainname', 'group']], on='dns_qdomainname', how='left')
    df.rename(columns={'group': bin_label}, inplace=True)
    df.set_index('dns_qdomainname', inplace=True)

    return df

# Apply the binning to different columns
df = bin_and_merge(df, 'dns_qdomainname', 'num_clients', 'client_sum_group')
df = bin_and_merge(df, 'dns_qdomainname', 'num_responses', 'responses_sum_group')
df = bin_and_merge(df, 'dns_qdomainname', 'num_shieldid', 'shield_sum_group')

In [4]:
import time
start_time = time.time()

# list the sum groups
sum_groups = [
    ("client_sum_group", "num_clients", "prob_obs_num_clients"),
    ("responses_sum_group", "num_responses", "prob_obs_num_responses"),
    ("shield_sum_group", "num_shieldid", "prob_obs_num_shield")
]

for group_col, num_col, prob_col in sum_groups:
    # loop over each sum group
    for group_val in df[group_col].sort_values().unique():
        # empty dictionary
        dict_ = {}
        # slice the df to sum group
        case = df.loc[df[group_col]==group_val]
        # create the number of bins so we get discrete bins
        num_bins = case[num_col].max()
        # cumulative distribution of the empirical observations
        pds, bins = np.histogram(case[num_col], bins=num_bins, density=False)
        probs = list(1 - (pds/np.sum(pds)).cumsum())
        probs.insert(0,1)
        probs.pop(-1)
        probs = np.array(probs)
        # list the integer bin values
        bins = list(bins)
        for i in range(len(bins)):
            bins[i] = int(i)
        bins.pop(0)
        bins = np.array(bins)
        if len(bins) == len(probs):
            # create dictionary
            dict_ = dict(zip(bins, probs))
            # replace the values in the column with dictionary values using apply
            df.loc[df[group_col] == group_val, prob_col] = case[num_col].apply(lambda x: dict_.get(int(x), 0))
            
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Elapsed time: {elapsed_time} seconds")

Elapsed time: 71.61721658706665 seconds


In [5]:
df['total_prob'] = (df.prob_obs_num_shield + df.prob_obs_num_clients + df.prob_obs_num_responses)/3

In [6]:
df.sort_values(by='total_prob').head(50)

,day,num_clients,num_responses,num_shieldid,num_dns_qname,client_sum_group,responses_sum_group,shield_sum_group,prob_obs_num_clients,prob_obs_num_responses,prob_obs_num_shield,total_prob
dns_qdomainname,,,,,,,,,,,,
kontinentalist.com,2022-11-15,247,8818,11,1,292-295,9144-9281,24-24,0.000155,0.000339,0.000051,0.000182
afar.com,2022-10-25,256,6378,11,1,599-614,13959-14171,133-137,0.000135,0.000315,0.000107,0.000186
themarkup.org,2022-10-03,283,14070,13,1,407-415,17273-17576,51-51,0.000136,0.000344,0.000090,0.000190
punchdrink.com,2022-11-14,280,10932,12,1,388-394,12307-12489,41-41,0.000143,0.000342,0.000088,0.000191
inews.co.uk,2022-10-14,117,4685,10,3,344-349,7114-7182,90-92,0.000151,0.000318,0.000109,0.000193
zenhabits.net,2022-11-03,287,9579,12,1,416-422,11262-11436,42-42,0.000195,0.000300,0.000091,0.000195
itsnicethat.com,2022-12-14,53,3528,12,1,132-133,4468-4517,52-52,0.000142,0.000360,0.000095,0.000199
euronews.com,2022-11-15,270,6218,12,2,583-598,8785-8888,138-142,0.000138,0.000313,0.000150,0.000201
foreignaffairs.com,2022-11-03,253,7760,11,3,532-543,11081-11261,102-103,0.000137,0.000315,0.000156,0.000203
